In [ ]:
# Tulis Program Anda Disini!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Definisikan path kaggle json
# Sesuaikan dengan path anda
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/Machine_learning/"

In [ ]:
%cd /content/drive/My Drive/Machine_learning/modul4

/content/drive/My Drive/Machine_learning/modul4


In [ ]:
# cek API 
!ls

base  crop_disease_wise  dataset_mod4  kaggle.json


In [ ]:
# pengunduhan dataset
!kaggle datasets download -d tushartalukder/crop-disease-wise

 99% 812M/818M [00:12<00:00, 57.3MB/s]
100% 818M/818M [00:12<00:00, 69.4MB/s]


In [ ]:
# cek apakah sudah terunduh
!ls

base  crop_disease_wise  dataset_mod4  kaggle.json


In [ ]:
# proses ekstraksi dan hapus file zip
!unzip \*.zip &> /dev/null && rm *.zip

In [ ]:
!mkdir 'dataset_mod4'

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

base_dir = '/content/drive/My Drive/Machine_learning/'
split_dir = '/content/drive/My Drive/Machine_learning/'


splitfolders.ratio(base_dir, output=split_dir, 
                   seed=1337, ratio=(.8, .19, .01), 
                   group_prefix=None) # default values

In [ ]:
import os
base_train_dir = "/content/drive/My Drive/Machine_learning/"

train_dir = os.path.join(base_train_dir, 'train')
test_dir = os.path.join(base_train_dir, 'test')
val_dir = os.path.join(base_train_dir, 'val')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
import tensorflow as tf

train_gen = datagen.flow_from_directory(train_dir, 
                                        color_mode="rgb", 
                                        target_size=(150, 150), 
                                        shuffle=True,
                                        class_mode='categorical'
)

test_gen = datagen.flow_from_directory(test_dir, 
                                        color_mode="rgb", 
                                        target_size=(150, 150), 
                                        shuffle=True,
                                        class_mode='categorical'
)

val_gen = datagen.flow_from_directory(val_dir, 
                                        color_mode="rgb", 
                                        target_size=(150, 150), 
                                        shuffle=True,
                                        class_mode='categorical'
)

Found 6311 images belonging to 4 classes.
Found 81 images belonging to 4 classes.
Found 1498 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout, GlobalMaxPooling2D, BatchNormalization

In [ ]:
# Feature Extraction Layer
model = Sequential()

model.add(InputLayer(input_shape=[150,150,3]))
model.add(Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))

model.add(Dropout(0.2))
model.add(GlobalMaxPooling2D())
model.add(Flatten())

In [ ]:
# Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))

In [ ]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 150, 150, 16)      448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 75, 75, 16)       0         
 2D)                                                             
                                                                 
 batch_normalization_6 (Batc  (None, 75, 75, 16)       64        
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          (None, 75, 75, 32)        4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 38, 38, 32)       0         
 g2D)                                                            
                                                      

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.001), 
              loss='categorical_crossentropy', 
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
H = model.fit(train_gen, batch_size=32, epochs=50, validation_data=val_gen)

Epoch 1/50
198/198 [==============================] - 2334s 12s/step - loss: 0.1891 - acc: 0.9304 - val_loss: 0.7461 - val_acc: 0.6983
Epoch 2/50
 58/198 [=======>......................] - ETA: 1:07 - loss: 0.0390 - acc: 0.9871

UnknownError: ignored